In [1]:
# import libraries
# ================

# for date and time opeations
from datetime import datetime
# for file and folder operations
import os
# for regular expression opeations
import re
# for listing files in a folder
import glob
# for getting web contents
import requests 
# storing and analysing data
import pandas as pd
# for scraping web contents
from bs4 import BeautifulSoup
# regular expression
import re
# numerical analysis
import numpy as np

In [2]:
# get data
# ========

# link at which web data recides
link = 'https://www.worldometers.info/coronavirus/'
# get web data
req = requests.get(link)
# parse web data
soup = BeautifulSoup(req.content, "html.parser")

In [3]:
# find the table
# ==============
# our target table is the last table in the page

# get the table head
# table head may contain the column names, titles, subtitles
thead = soup.find_all('thead')[-1]
# print(thead)

# get all the rows in table head
# it usually have only one row, which has the column names
head = thead.find_all('tr')
# print(head)

# get the table tbody
# it contains the contents
tbody = soup.find_all('tbody')[0]
# print(tbody)

# get all the rows in table body
# each row is each state's entry
body = tbody.find_all('tr')
# print(body)

In [4]:
# get the table contents
# ======================

# container for header rows / column title
head_rows = []
# container for table body / contents
body_rows = []

# loop through the head and append each row to head
for tr in head:
    td = tr.find_all(['th', 'td'])
    row = [i.text for i in td]
    head_rows.append(row)
# print(head_rows)

# loop through the body and append each row to body
for tr in body:
    td = tr.find_all(['th', 'td'])
    row = [i.text for i in td]
    body_rows.append(row)
# print(head_rows)

In [5]:
# save contents in a dataframe
# ============================
    
# skip last 3 rows, it contains unwanted info
# head_rows contains column title
df_bs = pd.DataFrame(body_rows[:len(body_rows)-6], 
                     columns=head_rows[0])         

# Drop 'S. No.' column
# df_bs.drop('S. No.', axis=1, inplace=True)

# there are 36 states+UT in India
df_bs.head(5)

,#,"Country,Other",TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/\n1M pop\n,Population,Continent,1 Caseevery X ppl,1 Deathevery X ppl,1 Testevery X ppl
0,,\nNorth America\n,"2,774,792","+5,792","156,222",+807,"1,250,722","+5,369","1,367,848","19,422",,,,,,North America,\n,,
1,,\nSouth America\n,"1,817,322","+1,105","72,629",+47,"1,022,055","+15,307","722,638","12,536",,,,,,South America,\n,,
2,,\nEurope\n,"2,326,847",,"187,929",,"1,266,099",,"872,819","6,187",,,,,,Europe,\n,,
3,,\nAsia\n,"1,945,634","+4,960","49,155",+107,"1,238,473","+2,144","658,006","19,249",,,,,,Asia,\n,,
4,,\nAfrica\n,"318,321",,"8,371",,"153,242",,"156,708",502,,,,,,Africa,\n,,


In [6]:
# drop unwanted rows
df_bs = df_bs.iloc[8:, :-3].reset_index(drop=True)

# drop unwanted columns
df_bs = df_bs.drop('#', axis=1)

# first few rows
df_bs.head()

,"Country,Other",TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/\n1M pop\n,Population,Continent
0,USA,"2,388,153",,"122,610",,"1,002,929",,"1,262,614","16,510","7,216",370,"29,013,182","87,664","330,959,930",North America
1,Brazil,"1,111,348",,"51,407",,"594,104","+14,878","465,837","8,318","5,229",242,"2,510,717","11,814","212,525,202",South America
2,Russia,"592,280",,"8,206",,"344,416",,"239,658","2,300","4,059",56,"17,289,691","118,477","145,933,256",Europe
3,India,"440,685",+235,"14,015",,"248,190",+53,"178,480","8,944",319,10,"7,137,716","5,173","1,379,678,183",Asia
4,UK,"305,289",,"42,647",,N/A,N/A,N/A,330,"4,498",628,"8,029,757","118,296","67,878,273",Europe


In [7]:
# rename columns
df_bs.columns = ['Country/Region', 'TotalCases', 'NewCases', 'TotalDeaths', 'NewDeaths',
       'TotalRecovered', 'NewRecovered', 'ActiveCases', 'Serious,Critical',
       'Tot Cases/1M pop', 'Deaths/1M pop', 'TotalTests', 'Tests/1M pop',
       'Population', 'Continent']

# rearrange and subselect columns
df_bs = df_bs[['Country/Region', 'Continent', 'Population', 'TotalCases', 'NewCases', 'TotalDeaths', 'NewDeaths',
       'TotalRecovered', 'NewRecovered', 'ActiveCases', 'Serious,Critical',
       'Tot Cases/1M pop', 'Deaths/1M pop', 'TotalTests', 'Tests/1M pop' ]]

# first few rows
df_bs.head()

,Country/Region,Continent,Population,TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/1M pop
0,USA,North America,"330,959,930","2,388,153",,"122,610",,"1,002,929",,"1,262,614","16,510","7,216",370,"29,013,182","87,664"
1,Brazil,South America,"212,525,202","1,111,348",,"51,407",,"594,104","+14,878","465,837","8,318","5,229",242,"2,510,717","11,814"
2,Russia,Europe,"145,933,256","592,280",,"8,206",,"344,416",,"239,658","2,300","4,059",56,"17,289,691","118,477"
3,India,Asia,"1,379,678,183","440,685",+235,"14,015",,"248,190",+53,"178,480","8,944",319,10,"7,137,716","5,173"
4,UK,Europe,"67,878,273","305,289",,"42,647",,N/A,N/A,N/A,330,"4,498",628,"8,029,757","118,296"


In [8]:
who_region = {}

# African Region AFRO
afro = "Algeria, Angola, Cabo Verde, Congo, DRC, Eswatini, Sao Tome and Principe, Benin, South Sudan, Western Sahara, Congo (Brazzaville), Congo (Kinshasa), Cote d'Ivoire, Botswana, Burkina Faso, Burundi, Cameroon, Cape Verde, Central African Republic, Chad, Comoros, Ivory Coast, Democratic Republic of the Congo, Equatorial Guinea, Eritrea, Ethiopia, Gabon, Gambia, Ghana, Guinea, Guinea-Bissau, Kenya, Lesotho, Liberia, Madagascar, Malawi, Mali, Mauritania, Mauritius, Mozambique, Namibia, Niger, Nigeria, Republic of the Congo, Rwanda, São Tomé and Príncipe, Senegal, Seychelles, Sierra Leone, Somalia, South Africa, Swaziland, Togo, Uganda, Tanzania, Zambia, Zimbabwe"
afro = [i.strip() for i in afro.split(',')]
for i in afro:
    who_region[i] = 'Africa'
    
# Region of the Americas PAHO
paho = 'Antigua and Barbuda, Argentina, Bahamas, Barbados, Belize, Bermuda, Bolivia, Brazil, Canada, Chile, Colombia, Costa Rica, Cuba, Dominica, Dominican Republic, Ecuador, El Salvador, Grenada, Guatemala, Guyana, Haiti, Honduras, Jamaica, Mexico, Nicaragua, Panama, Paraguay, Peru, Saint Kitts and Nevis, Saint Lucia, Saint Vincent and the Grenadines, Suriname, Trinidad and Tobago, United States, US, USA, Uruguay, Venezuela'
paho = [i.strip() for i in paho.split(',')]
for i in paho:
    who_region[i] = 'Americas'

# South-East Asia Region SEARO
searo = 'Bangladesh, Bhutan, North Korea, India, Indonesia, Maldives, Myanmar, Burma, Nepal, Sri Lanka, Thailand, Timor-Leste'
searo = [i.strip() for i in searo.split(',')]
for i in searo:
    who_region[i] = 'South-East Asia'

# European Region EURO
euro = 'Albania, Andorra, Greenland, Kosovo, Holy See, Vatican City, Liechtenstein, Armenia, Czechia, Austria, Azerbaijan, Belarus, Belgium, Bosnia and Herzegovina, Bulgaria, Croatia, Cyprus, Czech Republic, Denmark, Estonia, Finland, France, Georgia, Germany, Greece, Hungary, Iceland, Ireland, Israel, Italy, Kazakhstan, Kyrgyzstan, Latvia, Lithuania, Luxembourg, Malta, Monaco, Montenegro, Netherlands, North Macedonia, Norway, Poland, Portugal, Moldova, Romania, Russia, San Marino, Serbia, Slovakia, Slovenia, Spain, Sweden, Switzerland, Tajikistan, Turkey, Turkmenistan, Ukraine, United Kingdom, UK, Uzbekistan'
euro = [i.strip() for i in euro.split(',')]
for i in euro:
    who_region[i] = 'Europe'

# Eastern Mediterranean Region EMRO
emro = 'Afghanistan, Bahrain, Djibouti, Egypt, Iran, Iraq, Jordan, Kuwait, Lebanon, Libya, Morocco, Oman, Pakistan, Palestine, West Bank and Gaza, Qatar, Saudi Arabia, Somalia, Sudan, Syria, Tunisia, United Arab Emirates, UAE, Yemen'
emro = [i.strip() for i in emro.split(',')]
for i in emro:
    who_region[i] = 'Eastern Mediterranean'

# Western Pacific Region WPRO
wpro = 'Australia, Brunei, Cambodia, China, Cook Islands, Fiji, Japan, Hong Kong, Kiribati, Laos, Malaysia, Marshall Islands, Micronesia, Mongolia, Nauru, New Zealand, Niue, Palau, Papua New Guinea, Philippines, South Korea, S. Korea, Samoa, Singapore, Solomon Islands, Taiwan, Taiwan*, Tonga, Tuvalu, Vanuatu, Vietnam'
wpro = [i.strip() for i in wpro.split(',')]
for i in wpro:
    who_region[i] = 'Western Pacific'

In [9]:
# addding 'WHO Region' column
df_bs['WHO Region'] = df_bs['Country/Region'].map(who_region)

# countries with missing "WHO Region"
df_bs[df_bs['WHO Region'].isna()]['Country/Region'].unique()

array(['CAR', 'French Guiana', 'Mayotte', 'Diamond Princess',
       'Channel Islands', 'Réunion', 'Isle of Man', 'Martinique',
       'Cayman Islands', 'Faeroe Islands', 'Gibraltar', 'Guadeloupe',
       'Brunei ', 'Aruba', 'Sint Maarten', 'French Polynesia', 'Macao',
       'Saint Martin', 'St. Vincent Grenadines', 'Curaçao',
       'New Caledonia', 'Turks and Caicos', 'Falkland Islands',
       'Montserrat', 'MS Zaandam'], dtype=object)

In [10]:
# fix data
for col in df_bs.columns[2:]:
    # replace comma with empty string
    df_bs[col] = df_bs[col].str.replace('[,+ ]', '', regex=True)
    # replace 'N/A' with empty string
    df_bs[col] = df_bs[col].str.replace('N/A', '', regex=False)

# replace empty strings with np.nan
df_bs = df_bs.replace('', np.nan)

# first few rows
df_bs.head()

,Country/Region,Continent,Population,TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/1M pop,WHO Region
0,USA,North America,330959930,2388153,NaN,122610,NaN,1002929,NaN,1262614,16510,7216,370,29013182,87664,Americas
1,Brazil,South America,212525202,1111348,NaN,51407,NaN,594104,14878,465837,8318,5229,242,2510717,11814,Americas
2,Russia,Europe,145933256,592280,NaN,8206,NaN,344416,NaN,239658,2300,4059,56,17289691,118477,Europe
3,India,Asia,1379678183,440685,235,14015,NaN,248190,53,178480,8944,319,10,7137716,5173,South-EastAsia
4,UK,Europe,67878273,305289,NaN,42647,NaN,NaN,NaN,NaN,330,4498,628,8029757,118296,Europe


In [11]:
# save as .csv file
df_bs.to_csv('worldometer_data.csv', index=False)